# **Case Study** *(based on a true story)*

### Prepared by Daniel Tan, 2025


## 🧑‍💻 You're a **fresh graduate working as a Data Analyst** in your company.<br>

In [ ]:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/DanielTan18/pandas/refs/heads/main/assets/UM_student.png', width=500)

Just a few weeks in, your seniors are all coincidentally out of office, leaving only you to brace war.<br>
Miraculously, the Marketing Team Lead hands you a **huge csv file** of customer data, saying he wants to send a promotional email and SMS blast to only a certain portion of these customers. <br><br>
He then asks you to do the following:
- ✅ Filter these customers to ONLY those with open email domains (@gmail.com, @hotmail.com, @yahoo.com etc.)
- ✅ Additionally, customers' Phone Number, First Name & Last Name must not be empty.
- ✅ Also filter them to only customers who have a Subscription Date of 1st January 2020 onwards.
- ✅ Export the file in .csv format and give it to them in **ONE HOUR!**

If you filter the emails wrongly, there will be 😱 **legal consequences** 😱 as they are not allowed to email blast to their corporate customers (whatever that means).

# <b>Your time starts NOW! ⏲️</b>

# **1. Pandas Fundamentals**

In [ ]:
import pandas as pd

In [ ]:
file_path = "https://raw.githubusercontent.com/DanielTan18/pandas/refs/heads/main/data/dirty_customers.csv"

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.isna().sum()

# **2. Data Cleaning & Preprocessing**

## Filter these customers to ONLY open email domains

In [ ]:
df['Email Domain'] = df['Email'].str.split('@').str[-1]

# Count occurrences of each domain
domain_counts = df['Email Domain'].value_counts()
print(domain_counts)

In [ ]:
valid_domains = ("gmail.com", "yahoo.com", "hotmail.com", "aol.com", "live.com.my","rocketmail.com")

In [ ]:
df = df[df["Email"].str.endswith(valid_domains, na=False)]

In [ ]:
df.shape

## Primary Phone Number must not be Null

In [ ]:
invalid_phone = df[df["Phone 1"].str.contains(r"\.", na=False)] # captures anything with a "."

invalid_phone["Phone 1"]

## Validate Phone Number format

In [ ]:
import regex as re
import numpy as np

def clean_phone(phone):
    match = re.match(r"^\d{3}-?\d{3,4}\s?\d{3,4}$", phone)
    return phone if match else np.nan  # Keep valid numbers, remove invalid ones

df["Phone 1"] = df["Phone 1"].apply(clean_phone)
df = df.dropna(subset=["Phone 1"])  # Remove rows with invalid phone numbers

df

## First Name & Last Name must not be null

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna(subset=["First Name", "Last Name"])
df.isna().sum()

## Filter to customers whose Subscription Date is 1st January 2020 onwards.

In [ ]:
df["Subscription Date"] = pd.to_datetime(df["Subscription Date"], errors="coerce")

# Filter for dates on or after 2020-01-01
df = df[df["Subscription Date"] >= "2020-01-01"]

# Display filtered DataFrame
df

# Cuba try test if it's filtered correctly
# df[df["Subscription Date"] < "2020-01-01"]

## Tabulating your dataframe

In [ ]:
from tabulate import tabulate

print(tabulate(df, headers='keys', tablefmt='psql'))

Displaying some quick stats regarding gmail_customers

In [ ]:
print(df.describe())
print(len(df))

# 3. Data Analytics with Pandas

In [ ]:
df['Email Domain'] = df['Email'].str.split('@').str[-1]

# Count occurrences of each domain
domain_counts = df['Email Domain'].value_counts()

# Display the top 10 most common email domains
print("Top 10 Most Common Email Domains:")
print(domain_counts)

## Quick Data Visualisations with Plotly

In [ ]:
import plotly.express as px

# Convert value_counts to DataFrame
domain_counts_df = domain_counts.head(10).reset_index()
domain_counts_df.columns = ['Email Domain', 'Count']

# Create bar chart
fig = px.bar(domain_counts_df, x='Email Domain', y='Count',
             title="Customer Count Breakdown by Email Domain",
             text='Count', template='plotly_dark')

fig.show()


In [ ]:
import plotly.express as px

# Count occurrences of each city
city_counts = df['City'].value_counts()

city_df = city_counts.head(10).reset_index()
city_df.columns = ['City', 'Count']

# Create pie chart
fig = px.pie(city_df, names='City', values='Count',
             title="Top 10 Cities",
             template='plotly_dark',
             hole=0.2)  # Adjust hole size for a donut chart effect

fig.show()


In [ ]:
df.to_csv("clean_customers.csv", index=False)

...and there you have it! The Marketing Team Lead is satisfied with the results and your company is saved!

# Round of applause for yourselves and your knowledge of 🐼 Pandas 🐼! 👏👏👏

![congrats dumb.jpg](https://raw.githubusercontent.com/DanielTan18/pandas/refs/heads/main/assets/congrats_dumb.jpg)

In [ ]:
# @title Time for a break, you've all deserved it!
from IPython.display import Image
Image(url='https://media.tenor.com/MggLWlLRH98AAAAM/sleepy-sleepy-cat.gif')

# BONUS 1: DuckDB 🦆

In [ ]:
import duckdb

# Define input and output file paths
file_path = "https://raw.githubusercontent.com/DanielTan18/pandas/refs/heads/main/data/dirty_customers.csv"
output_csv = "duckdb_filtered_customers.csv"

# Define the DuckDB query
query = f"""
  COPY (
      WITH cleaned_data AS (
          SELECT *
          FROM read_csv_auto('{file_path}')
          WHERE
              COALESCE(email, '') != ''
              AND REGEXP_MATCHES(lower(email), '.*@(gmail\\.com|hotmail\\.com|yahoo\\.com|live\\.com\\.my|aol\\.com|rocketmail\\.com)')
              AND COALESCE("Phone 1", '') != ''
              AND REGEXP_MATCHES("Phone 1", '^\\d{{3}}-?\\d{{3,4}}\\s?\\d{{3,4}}$')
              AND COALESCE("First Name", '') != ''
              AND COALESCE("Last Name", '') != ''
              AND "Subscription Date" >= '2020-01-01'
      )
      SELECT * FROM cleaned_data
  )
  TO '{output_csv}' WITH (HEADER, DELIMITER ',');
"""


# Execute the query
duckdb.query(query)

print(f"Filtered data saved to {output_csv}")

In [ ]:
query2 = f"""
select count(distinct "customer id") FROM read_csv_auto('{output_csv}')
"""
duckdb.query(query2)

# BONUS 2: POLARS! 🐻‍❄️

In [ ]:
import polars as pl

file_path = "https://raw.githubusercontent.com/DanielTan18/pandas/refs/heads/main/data/dirty_customers.csv"
output_csv = "polars_filtered_customers.csv"
polar_df = pl.read_csv(file_path)

cleaned_data = polar_df.filter(
    (pl.col("Email").is_not_null() & pl.col("Email").str.contains(r".*@(gmail\.com|hotmail\.com|yahoo\.com|live\.com\.my|aol\.com|rocketmail\.com)", literal=False)) &
    (pl.col("Phone 1").is_not_null() & pl.col("Phone 1").str.contains(r"^\d{3}-?\d{3,4}\s?\d{3,4}$")) &
    (pl.col("First Name").is_not_null()) &
    (pl.col("Last Name").is_not_null()) &
    (pl.col("Subscription Date") >= "2020-01-01")
)

print(cleaned_data)
cleaned_data.write_csv(output_csv)

### **Quick Comparison of Pandas vs DuckDB vs Polars**

| **Feature/Aspect**    | **Pandas**                           | **DuckDB**                            | **Polars**                          |   |
|-------------------|----------------------------------|-----------------------------------|---------------------------------|---|
| *Type*              | DataFrame library                | In-process SQL database           | DataFrame library               |   |
|*Execution Model*   | Single-threaded                  | Multi-threaded (SQL-based)        | Multi-threaded                  |   |
| *Ease of Use*       | Very easy (Pythonic API)         | Moderate (requires SQL knowledge) | Easy (Pandas-like API)          |   |
| *Performance*       | Slow for large datasets          | Very fast (optimized for OLAP)    | Very fast (multi-threaded)      |   |
| *Memory Efficiency* | High memory usage                | Memory-efficient                  | Memory-efficient                |   |
| *SQL Support*       | No (requires external libraries) | Native SQL support                | No (but can integrate with SQL) |   |
| *Community*         | Large and mature                 | Growing                           | Growing                         |   |
| *Best For*          | Small to medium datasets, EDA    | Analytical queries, large data    | Large datasets, ETL pipelines   |   |

# Appendix: Dataset Generation

In [ ]:
import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv(file_path)
print(df)

# Define email domain probabilities
email_domains = {
    "@gmail.com": 80.23,
    "@yahoo.com": 10.77,
    "@hotmail.com": 8.91,
    "@aol.com": 2.0,
    "@live.com.my": 1.5,
    "@rocketmail.com": 1.0,
    "@assenture.com.my": 0.5,
    "@peeuuc.com.my": 0.5,
    "@lionmonies.com.my": 0.3,
    "@gulugol.com.my": 0.2,
    "@pehdronas.com.my": 0.1,
}

# Normalize probabilities
email_domains_list = []
for domain, weight in email_domains.items():
    email_domains_list.extend([domain] * int(weight * 10))  # Expand based on percentage

# Modify email addresses
df["Email"] = df["First Name"].str.lower() + "." + df["Last Name"].str.lower() + np.random.choice(email_domains_list, size=len(df))

# List of Malaysian cities
import numpy as np

# List of Malaysian cities with priority
malaysian_cities = [
    "Kuala Lumpur", "Petaling Jaya", "George Town", "Johor Bahru", "Kota Kinabalu",
    "Ipoh", "Shah Alam", "Kuching", "Seremban", "Melaka", "Alor Setar", "Kuantan",
    "Miri", "Sibu", "Sandakan", "Taiping", "Bintulu", "Batu Pahat", "Tawau", "Kangar"
]

# Approximate probability distribution (sum should be 1)
city_probabilities = [
    0.42,  # Kuala Lumpur (~42%)
    0.12,  # Petaling Jaya (~15%)
    0.11,  # George Town (~12%)
    0.09,  # Johor Bahru (~10%)
    0.08,  # Kota Kinabalu (~8%)
    0.03,  # Ipoh (~3%)
    0.02,  # Shah Alam (~2%)
    0.02,  # Kuching (~2%)
    0.02,  # Seremban (~1%)
    0.02,  # Melaka (~1%)
    0.01,  # Alor Setar (~1%)
    0.02,  # Kuantan (~1%)
    0.007, # Miri (~0.7%)
    0.007, # Sibu (~0.7%)
    0.007, # Sandakan (~0.5%)
    0.005, # Taiping (~0.5%)
    0.004, # Bintulu (~0.4%)
    0.004, # Batu Pahat (~0.4%)
    0.003, # Tawau (~0.3%)
    0.003  # Kangar (~0.3%)
]

# Assign cities based on weighted probabilities
df["City"] = np.random.choice(malaysian_cities, size=len(df), p=city_probabilities)

df["Country"] = "Malaysia"
df = df.drop('Phone 1', axis=1)

# Save modified CSV
df.to_csv(f"/content/drive/My Drive/Colab Notebooks/customer_modified.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
import random
import re

# Load the original dataset
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/customer_modified.csv")

# Introduce null and empty values in random places
def introduce_nulls(df, columns, null_percentage=0.05, empty_percentage=0.03):
    total_rows = len(df)
    for col in columns:
        null_indices = np.random.choice(df.index, size=int(total_rows * null_percentage), replace=False)
        empty_indices = np.random.choice(df.index, size=int(total_rows * empty_percentage), replace=False)

        df.loc[null_indices, col] = np.nan  # Set to null
        df.loc[empty_indices, col] = ""  # Set to empty string

introduce_nulls(df, ["First Name", "Last Name", "Company", "City", "Phone 1", "Email"], 0.03, 0.01)

def generate_valid_malaysian_phone():
    # Different valid prefixes based on Malaysian phone formats
    prefixes = [
        "01{}-{} {}",   # Mobile numbers (01x-xxx xxxx)
        "01{}-{} {}",   # Mobile numbers (01x-xxxx xxxx)
    ]

    # Choose a format
    format_choice = random.choice(prefixes)

    # Generate random digits for different parts of the phone number
    first_digit = random.choice([str(d) for d in range(10) if d != 5])  # 0-9 except 5
    mid_3_digits = random.randint(100, 999)  # Middle section (3 digits)
    last_4_digits = random.randint(1000, 9999)  # Last section (4 digits)

    # Format the phone number accordingly
    return format_choice.format(first_digit, mid_3_digits, last_4_digits)

# Example usage:
for _ in range(10):
    print(generate_valid_malaysian_phone())


def generate_invalid_malaysian_phone():
    formats = [
        f"0{random.randint(0,9)}{random.randint(0,9)}-{random.randint(100,999)} {random.randint(1000,9999)}",  # Wrong prefix
        f"011-{random.randint(1000,9999)}-{random.randint(1000,9999)}",  # Extra digit segment
        f"01{random.randint(0,9)}.{random.randint(10,99)}.{random.randint(1000,9999)}",  # Wrong separator (dots)
        f"011{random.randint(100000000,999999999)}",  # No separators
        f"011-{random.randint(10,99)}-{random.randint(100,999)}-{random.randint(1000,9999)}"  # Extra hyphens
    ]
    return random.choice(formats)

# Introduce a mix of valid and invalid phone numbers
df["Phone 1"] = [generate_valid_malaysian_phone() if random.random() > 0.2 else generate_invalid_malaysian_phone() for _ in range(len(df))]

# Save the manipulated dataset
df.to_csv("/content/drive/My Drive/Colab Notebooks/dirty_customers.csv", index=False)
